<a href="https://colab.research.google.com/github/luiscunhacsc/udemy-ia-pt/blob/main/parte1b_visao/parte1b_exemplo02_vgg16_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Neste exemplo, vamos aprender como usar modelos pré-treinados para classificação de imagens. Vamos utilizar o conjunto de dados "CIFAR-10", que consiste em 60.000 imagens a cores de 32x32 divididas em 10 classes diferentes.

Começaremos utilizando um modelo pré-treinado diretamente para classificar imagens. Em seguida, vamos realizar o "freezing" e o "fine-tuning" do modelo para adaptá-lo melhor ao nosso conjunto de dados. Por fim, vamos explorar algumas técnicas adicionais, como o "data augmentation" e a regularização.

In [ ]:
# Importação das bibliotecas necessárias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

# Carregar o conjunto de dados CIFAR-10

O conjunto de dados CIFAR-10 contém 60.000 imagens a cores de 32x32, divididas em 10 classes, com 6.000 imagens por classe. Existem 50.000 imagens de treino e 10.000 imagens de teste.

In [ ]:
# Carregar o conjunto de dados
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalizar os dados
train_images = train_images / 255.0
test_images = test_images / 255.0

# Converter os rótulos para one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Uso de um modelo pré-treinado

Vamos começar usando o modelo VGG16, que é pré-treinado no conjunto de dados ImageNet, para classificar as imagens CIFAR-10. Usaremos o modelo diretamente sem qualquer modificação.

In [ ]:
# Célula 6: Usar o modelo VGG16 pré-treinado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Adicionar camadas de classificação personalizadas
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Construir o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Avaliar o desempenho do modelo pré-treinado no conjunto de teste
evaluation = model.evaluate(test_images, test_labels)
print(f"Acurácia do modelo pré-treinado no conjunto de teste: {evaluation[1]}")


# Fine-tuning do modelo pré-treinado

Agora, vamos realizar o "fine-tuning" do modelo pré-treinado. Inicialmente, vamos congelar todas as camadas do

modelo base (VGG16) e treinar apenas as camadas superiores que adicionamos. Isto é chamado de "feature extraction". Depois, vamos descongelar algumas das camadas do modelo base e realizar um ajuste fino em conjunto com as camadas adicionadas.

In [ ]:
# Feature extraction - congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_feature_extraction = model.fit(train_images, train_labels, batch_size=128, epochs=5, validation_data=(test_images, test_labels))

# Descongelar algumas camadas e realizar ajuste fino

Agora vamos descongelar algumas das camadas do modelo base e realizar o ajuste fino em conjunto com as camadas adicionadas.

In [ ]:
# Descongelar camadas e realizar ajuste fino
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_fine_tuning = model.fit(train_images, train_labels, batch_size=128, epochs=5, validation_data=(test_images, test_labels))

# Data Augmentation

O "Data Augmentation" aumenta o tamanho do conjunto de treino através da criação de versões modificadas das imagens, como rotações, deslocamentos e zooms. Isto pode ajudar a melhorar o desempenho e a generalização do modelo.

In [ ]:
# Utilizar Data Augmentation
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.2)
])

inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

# Construir o modelo final com Data Augmentation
model_with_augmentation = Model(inputs=inputs, outputs=outputs)

# Compilar o modelo
model_with_augmentation.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_with_augmentation = model_with_augmentation.fit(train_images, train_labels, batch_size=128, epochs=5, validation_data=(test_images, test_labels))

# Visualizar os resultados

Vamos agora visualizar o desempenho dos modelos em diferentes fases: uso direto do modelo pré-treinado, após "feature extraction", após ajuste fino e após "data augmentation".

In [ ]:
# Visualização dos resultados
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_feature_extraction.history['val_accuracy'], label='Feature Extraction')
plt.plot(history_fine_tuning.history['val_accuracy'], label='Fine-tuning')


plt.plot(history_with_augmentation.history['val_accuracy'], label='Data Augmentation')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Desempenho do Modelo')

plt.subplot(1, 2, 2)
plt.plot(history_feature_extraction.history['val_loss'], label='Feature Extraction')
plt.plot(history_fine_tuning.history['val_loss'], label='Fine-tuning')
plt.plot(history_with_augmentation.history['val_loss'], label='Data Augmentation')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.title('Desempenho do Modelo')

plt.show()

# Conclusão

Neste exemplo, explorámos diferentes técnicas para aproveitar os modelos pré-treinados para classificação de imagens. Utilizámos o conjunto de dados CIFAR-10 e o modelo VGG16 pré-treinado.

Começámos utilizando o modelo diretamente, depois fizemos "feature extraction", seguido de ajuste fino e, finalmente, utilizámos "data augmentation".

É importante notar que escolher as melhores técnicas e hiperparâmetros pode depender do conjunto de dados e do problema específico. Experimentar e avaliar diferentes abordagens é uma parte importante do processo de desenvolvimento de modelos de deep learning.